In [1]:
import pyaudio
import wave
import speech_recognition as sr
import read_wav_files
import glob
import time
import os 
import pandas as pd
import threading
import re
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import word_tokenize

In [2]:
r = sr.Recognizer()

i = 0
count = 0
df = pd.DataFrame(columns = ['Text_Said'])  
stop_words = ["a","an","the","is","am",
              "are","was","were","been",
              "has","have","too","had",
              "do","be","does","did","as",
              "for","nor","by","further",
              "into","these"]

In [3]:
class RecordingThread(object):
    
    def __init__(self, interval=0):
        
        self.interval = interval
        thread = threading.Thread(target=self.run, args=())
        thread.daemon = True
        thread.start()
        

    def run(self):
        
        while True:
            global count
            count=count+1
            CHUNK = 1024
            FORMAT = pyaudio.paInt16
            CHANNELS = 2
            RATE = 44100
            RECORD_SECONDS = 5
            WAVE_OUTPUT_FILENAME = 'output'+str(count)+'.wav'
            
            p = pyaudio.PyAudio()

            stream = p.open(format=FORMAT,
                            channels=CHANNELS,
                            rate=RATE,
                            input=True,
                            frames_per_buffer=CHUNK)


            frames = []

            for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
                data = stream.read(CHUNK)
                frames.append(data)

            stream.stop_stream()
            stream.close()
            p.terminate()

            wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
            wf.setnchannels(CHANNELS)
            wf.setsampwidth(p.get_sample_size(FORMAT))
            wf.setframerate(RATE)
            wf.writeframes(b''.join(frames))
            wf.close()

            time.sleep(self.interval)


In [4]:
def speech_to_text(text):
    
        try:
            global df
            global i
            global lst 
            lst = []
            
            lst.append(text['alternative'][0]['transcript'])
            df['Text_Said'] = lst
            df['Text_Tokenized']= df['Text_Said'].apply(lambda x: word_tokenize(x))
            df['No_Stopwords']= df['Text_Tokenized'].apply(lambda x:[word for word in x if word not in stop_words])
            df['Text_Lemmatized']=df['No_Stopwords'].apply(lambda x:[lemmatizer.lemmatize(token) for token in x])
            
            df.to_csv('file'+str(i)+'.csv', header=True, index=False)
            i=i+1
            lst=[]
            df = pd.DataFrame(columns = ['Text_Said']) #clear it out to start again
            
        except TypeError:
            pass

In [5]:
rth = RecordingThread()
while True:

    try:
        file_name = glob.glob("output*")
        if len(file_name)!=0:
            #print("dealing with file :",file_name[0])
            text=read_wav_files.record_recognize_audio(file_name[0])
            print("----------------------------------------------------")
            print(text['alternative'][0]['transcript'])
            print("----------------------------------------------------")
            speech_to_text(text)
            os.remove(file_name[0])
            
        else:
            continue
    except ValueError:
        continue
    except TypeError:
        continue
        

----------------------------------------------------
hello how are you are you here
----------------------------------------------------
----------------------------------------------------
thank you for your calling I will do that
----------------------------------------------------
----------------------------------------------------
please can you show that
----------------------------------------------------
----------------------------------------------------
I'm so happy for that this is a good thing for you
----------------------------------------------------
----------------------------------------------------
I think I will be happy to listen
----------------------------------------------------
----------------------------------------------------
1234567
----------------------------------------------------
----------------------------------------------------
19 123456
----------------------------------------------------
----------------------------------------------------
Anto

KeyboardInterrupt: 